In [14]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata

In [ ]:
# full_raw = pd.read_csv('/Users/javier/VSCODE/datahub/enron_data_fraud_labeled.csv') 

# filtered_raw = full_raw['Body'].iloc[:100_000]

# filtered_raw.to_csv('test.csv')

In [ ]:
filtered_raw = pd.read_csv('local/DSA4213_vsc/final_project/test.csv')

In [ ]:
test = "Hi John,  Please check out our new product at https://www.example.com/special-offer.  You can also visit www.testsite.org for more details.  I've attached the latest report as quarterly_results.pdf and also a backup copy as report.docx send to abc@gmail.com and john.doe@gmail.org. Let me know if you have trouble opening summary.xlsx.  Best,  Alice"

html_text = """
<html>
  <head>
    <title>Quarterly Update</title>
  </head>
  <body>
    <h1>Special Offer!</h1>
    <p>Dear customer,</p>
    
    <p>
      Please download the latest reports:
      <a href="https://example.com/files/quarterly_report.pdf">Quarterly Report</a>,
      <a href="https://example.com/files/summary.docx">Summary</a>,
      and <a href="https://example.com/files/data.xlsx">Data File</a>.
    </p>

    <p>
      If you cannot access the files, please email 
      <a href="mailto:support@example.com">support@example.com</a>.
    </p>

    <p>
      Alternatively, you may contact John at john.doe@workmail.org or visit our site 
      <a href="http://www.testsite.org">www.testsite.org</a>.
    </p>

    <p>
      Attached reference documents: <b>budget_2024.pdf</b>, <b>plan_final.docx</b>
    </p>

    <p>
      loveeeeeeeeee
      lovee33333eeee
      a-p-p-l-e
      b.a.n.a.n.a
      fr33 c4$h 
    </p
  </body>
</html>
"""


In [6]:
# masking special token
def mask_tokens(text):
    # replace URLs (http, https, www)
    text = re.sub(r'(https?://\S+|www\.\S+)', '<URL>', text)

    # replace common file extensions (customize list)
    text = re.sub(r'\b\w+\.(pdf|docx|xlsx|txt|csv|tar\.gz)\b', '<FILE>', text)

    # emails
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '<EMAIL>', text)

    return text



"Hi John,  Please check out our new product at <URL>  You can also visit <URL> for more details.  I've attached the latest report as <FILE> and also a backup copy as <FILE> send to <EMAIL> and <EMAIL>. Let me know if you have trouble opening <FILE>.  Best,  Alice"

In [ ]:
# un HTML raw text 
def strip_html(raw_html):
    """
    Strip HTML tags, scripts, styles, and normalize whitespace
    to return clean raw text from HTML emails.
    """
    soup = BeautifulSoup(raw_html, "html.parser")

    
    for a in soup.find_all("a"):
        a_attribute = a['href'] 

        a_attribute = mask_tokens(a_attribute)

        if a_attribute == '<URL>' : 
            a.replace_with('<URL>')

        elif a_attribute =='<EMAIL>' : 
            a.replace_with('<EMAIL>')
        
        elif a_attribute == '<FILE>' : 
            a.replace_with('<FILE>')

    # remove script, style, head, and metadata tags
    for tag in soup(["script", "style", "head", "title", "meta", "[document]"]):
        tag.decompose()

    # extract text
    text = soup.get_text(separator=" ")

    # normalize unicode 
    text = unicodedata.normalize("NFKC", text)

    # replace non-breaking spaces specifically (unicode)
    text = text.replace("\xa0", " ")

    # collapse all whitespace tokens (line breaks, tabs, multiple spaces) into one space and remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # rim leading/trailing spaces
    return text

In [59]:
# special case handling
mapper = str.maketrans({
    '0':'o','1':'l','3':'e','4':'a','5':'s','7':'t','$':'s','@':'a'
})

def deobfuscate_words(text):
    # replace text to number 
    text = text.translate(mapper)
    # remove weird spaces etc 
    text = re.sub(r'(?i)(?<=\w)[^A-Za-z0-9\s]{1,3}(?=\w)', '', text)
    return text

def word_capper(text):
    text = re.sub(r'(.)\1{' + str(2) + r',}', lambda m: m.group(1)*2, text)
    text = re.sub(r'([!?.,])\1{1,}', r'\1\1', text)
    return text

In [ ]:
raw = html_text
raw = strip_html(raw)
raw = mask_tokens(raw)
raw = deobfuscate_words(raw)
raw = word_capper(raw)


print(raw)

Special Offer! Dear customer, Please download the latest reports: <URL> , <URL> , and <URL> . If you cannot access the files, please email <EMAIL> . Alternatively, you may contact John at <EMAIL> or visit our site <URL> . Attached reference documents: <FILE> , <FILE> lovee lovee apple banana free cash


In [ ]:
# whitelist filtering

In [ ]:
# full call